In [11]:
import torch
import cv2
import numpy as np
import easyocr
import pytesseract
from pathlib import Path
from collections import Counter
import sys

# Yolov5 klasörünü ekle
sys.path.insert(0, './yolov5')

from models.common import DetectMultiBackend
from utils.general import non_max_suppression, scale_boxes
from utils.plots import save_one_box

# OCR modelleri yükle
reader = easyocr.Reader(['en'])

# Model yükle (YOLOv5s)
from utils.torch_utils import select_device

device = select_device('')
model = DetectMultiBackend('yolov5s.pt', device=device)

stride, names = model.stride, model.names
imgsz = (640, 640)

# Test görüntüyü yükle
img0 = cv2.imread('Ekran görüntüsü 2025-03-31 152531.png')
imgsz = (640, 640)  # veya (512, 512), (416, 416), (320, 320) gibi
img = cv2.resize(img0.copy(), imgsz, interpolation=cv2.INTER_LINEAR)

img_tensor = torch.from_numpy(img).to(device).permute(2, 0, 1).float() / 255.0
img_tensor = img_tensor.unsqueeze(0)

# Tahmin yap
pred = model(img_tensor, augment=False, visualize=False)
pred = non_max_suppression(pred, conf_thres=0.4, iou_thres=0.5)[0]

# Sonuçları işle
if pred is not None and len(pred):
    pred[:, :4] = scale_boxes(img_tensor.shape[2:], pred[:, :4], img0.shape).round()

    for det in pred:
        x1, y1, x2, y2, conf, cls = map(int, det)
        torso_crop = save_one_box(det[:4], img0, save=False)

        # Opsiyonel: Görüntü iyileştirme (basit sharpening)
        torso_crop = cv2.detailEnhance(torso_crop, sigma_s=10, sigma_r=0.15)

        # OCR Aşamaları
        easy_result = reader.readtext(torso_crop)
        easy_text = easy_result[0][1] if easy_result else ""

        tesseract_text = pytesseract.image_to_string(torso_crop, config='--psm 7 -c tessedit_char_whitelist=0123456789')
        tesseract_text = tesseract_text.strip()

        # (Opsiyonel) parseq_text = call_parseq_model(torso_crop)  # Model kuruluysa

        results = [easy_text, tesseract_text]
        most_common = Counter(results).most_common(1)[0][0]

        print(f"✅ Detected Jersey Number: {most_common}")
else:
    print("No players detected.")


YOLOv5  v7.0-411-gf4d8a84c Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs


✅ Detected Jersey Number: 
✅ Detected Jersey Number: 
✅ Detected Jersey Number: 2
✅ Detected Jersey Number: KN
✅ Detected Jersey Number: 
✅ Detected Jersey Number: 
✅ Detected Jersey Number: 
✅ Detected Jersey Number: 
✅ Detected Jersey Number: 
✅ Detected Jersey Number: 


In [13]:
import cv2
import torch
import numpy as np
import easyocr
import pytesseract
from pathlib import Path
from collections import Counter
import sys
from utils.torch_utils import select_device
from utils.plots import save_one_box
from utils.general import non_max_suppression, scale_boxes
from models.common import DetectMultiBackend

# OCR başlat
reader = easyocr.Reader(['en'])
pytesseract.pytesseract.tesseract_cmd = 'tesseract'

# Cihaz seç
device = select_device('')
model = DetectMultiBackend('yolov5s.pt', device=device)
model.eval()
imgsz = (640, 640)

# Görüntü oku ve resize et
img0 = cv2.imread('Ekran görüntüsü 2025-03-31 152531.png')
img = cv2.resize(img0.copy(), imgsz)
img_resized = img[:, :, ::-1].transpose(2, 0, 1)  # BGR→RGB, HWC→CHW
img_tensor = torch.from_numpy(np.ascontiguousarray(img_resized)).float() / 255.0
img_tensor = img_tensor.unsqueeze(0).to(device)

# YOLOv5 ile tahmin
with torch.no_grad():
    pred = model(img_tensor)[0]
    pred = non_max_suppression(pred, conf_thres=0.4, iou_thres=0.5)[0]

# Tahmin varsa işleyelim
if pred is not None and len(pred):
    pred[:, :4] = scale_boxes(img_tensor.shape[2:], pred[:, :4], img0.shape).round()

    for *xyxy, conf, cls in pred:
        # Oyuncu kutusunu çiz
        xyxy = torch.tensor(xyxy).view(1, 4)
        crop = save_one_box(xyxy, img0, save=False)

        # Opsiyonel iyileştirme
        crop = cv2.detailEnhance(crop, sigma_s=10, sigma_r=0.15)

        # OCR (ensemble)
        easy_result = reader.readtext(crop)
        easy_text = easy_result[0][1] if easy_result else ""

        tess_text = pytesseract.image_to_string(crop, config='--psm 7 -c tessedit_char_whitelist=0123456789').strip()

        # Sonuçları birleştir
        results = [easy_text, tess_text]
        jersey_number = Counter(results).most_common(1)[0][0]

        # Görsel üzerine çizim yap
        x1, y1, x2, y2 = map(int, xyxy.squeeze())
        cv2.rectangle(img0, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img0, f'#{jersey_number}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        print(f"✅ Tespit: {jersey_number}")

# Sonuç göster
import matplotlib.pyplot as plt

# BGR → RGB
img_rgb = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10, 8))
plt.imshow(img_rgb)
plt.title("Detected Players and Jersey Numbers")
plt.axis("off")
plt.show()

# İsteğe bağlı kaydet
cv2.imwrite("output_detected.jpg", img0)


YOLOv5  v7.0-411-gf4d8a84c Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs


✅ Tespit: 
✅ Tespit: 
✅ Tespit: BENV 66
✅ Tespit: 2
✅ Tespit: 
✅ Tespit: 
✅ Tespit: 
✅ Tespit: 
✅ Tespit: 
✅ Tespit: 
✅ Tespit: 
✅ Tespit: 
✅ Tespit: 
✅ Tespit: 
✅ Tespit: 


True

In [14]:
# Gelişmiş Forma Numarası Tespiti Pipeline v2

import cv2
import torch
import numpy as np
import easyocr
import pytesseract
import re
from collections import Counter
from pathlib import Path
import matplotlib.pyplot as plt
import sys

# YOLOv5 klasörünü ekle
sys.path.insert(0, './yolov5')
from models.common import DetectMultiBackend
from utils.general import non_max_suppression, scale_boxes
from utils.torch_utils import select_device
from utils.plots import save_one_box

# OCR modelleri
reader = easyocr.Reader(['en'])
pytesseract.pytesseract.tesseract_cmd = 'tesseract'

# Cihaz seçimi ve model yükleme
device = select_device('')
model = DetectMultiBackend('yolov5s.pt', device=device)
model.eval()
imgsz = (640, 640)

# Yardımcı fonksiyonlar
def clean_ocr_result(text):
    num = re.sub(r"[^0-9]", "", text)
    if 1 <= len(num) <= 2:
        return num
    return ""

def is_valid_box(x1, y1, x2, y2, img_height):
    w, h = x2 - x1, y2 - y1
    if h < 40 or w < 20 or h/w > 6:
        return False
    if y2 < img_height * 0.3:
        return False  # Üst kısımlar genellikle yazı olur
    return True

def crop_lower_half(box):
    x1, y1, x2, y2 = map(int, box)
    h = y2 - y1
    return [x1, y1 + int(h * 0.5), x2, y2]

# Görüntü yükle ve resize et
img0 = cv2.imread('Ekran görüntüsü 2025-03-31 152531.png')
img = cv2.resize(img0.copy(), imgsz)
img_resized = img[:, :, ::-1].transpose(2, 0, 1)
img_tensor = torch.from_numpy(np.ascontiguousarray(img_resized)).float() / 255.0
img_tensor = img_tensor.unsqueeze(0).to(device)

# Tahmin
with torch.no_grad():
    pred = model(img_tensor)[0]
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.5)[0]

# Tahmin varsa işle
if pred is not None and len(pred):
    pred[:, :4] = scale_boxes(img_tensor.shape[2:], pred[:, :4], img0.shape).round()

    for *xyxy, conf, cls in pred:
        x1, y1, x2, y2 = map(int, xyxy)
        if not is_valid_box(x1, y1, x2, y2, img0.shape[0]):
            continue

        # Alt yarıyı crop et
        crop_box = crop_lower_half([x1, y1, x2, y2])
        torso_crop = img0[crop_box[1]:crop_box[3], crop_box[0]:crop_box[2]]

        # OCR
        easy_result = reader.readtext(torso_crop)
        easy_text = easy_result[0][1] if easy_result else ""

        tess_text = pytesseract.image_to_string(
            torso_crop, config='--psm 7 -c tessedit_char_whitelist=0123456789')

        # Ensemble ve temizleme
        results = [easy_text, tess_text.strip()]
        jersey_number = Counter(map(clean_ocr_result, results)).most_common(1)[0][0]

        # Çizim
        cv2.rectangle(img0, (x1, y1), (x2, y2), (0, 255, 0), 2)
        if jersey_number:
            cv2.putText(img0, f'#{jersey_number}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            print(f"✅ Tespit: {jersey_number}")

# Görseli göster (Matplotlib)
img_rgb = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(12, 8))
plt.imshow(img_rgb)
plt.axis("off")
plt.title("Detected Players and Jersey Numbers")
plt.show()

# Sonuçu kaydet
cv2.imwrite("output_detected1.jpg", img0)

YOLOv5  v7.0-411-gf4d8a84c Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs


True

In [ ]:
# Gelişmiş Forma Numarası Tespiti Pipeline v2 (PARSeq destekli)

import cv2
import torch
import numpy as np
import easyocr
import pytesseract
import re
from collections import Counter
from pathlib import Path
import matplotlib.pyplot as plt
import sys
from torchvision import transforms
from PIL import Image

# YOLOv5 klasörü
sys.path.insert(0, './yolov5')
from models.common import DetectMultiBackend
from utils.general import non_max_suppression, scale_boxes
from utils.torch_utils import select_device
from utils.plots import save_one_box

# OCR modelleri
reader = easyocr.Reader(['en'])
pytesseract.pytesseract.tesseract_cmd = 'tesseract'

# PARSeq modeli
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
parseq_model = VisionEncoderDecoderModel.from_pretrained("facebook/parseq")
parseq_processor = TrOCRProcessor.from_pretrained("facebook/parseq")
parseq_model.eval()

# Cihaz seçimi ve model yükleme
device = select_device('')
model = DetectMultiBackend('yolov5s.pt', device=device)
model.eval()
imgsz = (640, 640)

# Yardımcı fonksiyonlar
def clean_ocr_result(text):
    num = re.sub(r"[^0-9]", "", text)
    if 1 <= len(num) <= 2:
        return num
    return ""

def is_valid_box(x1, y1, x2, y2, img_height):
    w, h = x2 - x1, y2 - y1
    if h < 40 or w < 20 or h/w > 6:
        return False
    if y2 < img_height * 0.3:
        return False
    return True

def crop_lower_half(box):
    x1, y1, x2, y2 = map(int, box)
    h = y2 - y1
    return [x1, y1 + int(h * 0.35), x2, y2]  # alt %65'i al

def run_parseq(img_crop):
    pil_img = Image.fromarray(cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB))
    inputs = parseq_processor(images=pil_img, return_tensors="pt")
    with torch.no_grad():
        generated_ids = parseq_model.generate(**inputs)
    text = parseq_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return text.strip()

# Görüntü yükle
img0 = cv2.imread('Ekran görüntüsü 2025-03-31 152531.png')
img = cv2.resize(img0.copy(), imgsz)
img_resized = img[:, :, ::-1].transpose(2, 0, 1)
img_tensor = torch.from_numpy(np.ascontiguousarray(img_resized)).float() / 255.0
img_tensor = img_tensor.unsqueeze(0).to(device)

# Tahmin
with torch.no_grad():
    pred = model(img_tensor)[0]
    pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.5)[0]

if pred is not None and len(pred):
    pred[:, :4] = scale_boxes(img_tensor.shape[2:], pred[:, :4], img0.shape).round()

    for *xyxy, conf, cls in pred:
        x1, y1, x2, y2 = map(int, xyxy)
        if not is_valid_box(x1, y1, x2, y2, img0.shape[0]):
            continue

        crop_box = crop_lower_half([x1, y1, x2, y2])
        torso_crop = img0[crop_box[1]:crop_box[3], crop_box[0]:crop_box[2]]

        easy_result = reader.readtext(torso_crop)
        easy_text = easy_result[0][1] if easy_result else ""

        tess_text = pytesseract.image_to_string(
            torso_crop, config='--psm 7 -c tessedit_char_whitelist=0123456789').strip()

        parseq_text = run_parseq(torso_crop)

        results = [easy_text, tess_text, parseq_text]
        jersey_number = Counter(map(clean_ocr_result, results)).most_common(1)[0][0]

        cv2.rectangle(img0, (x1, y1), (x2, y2), (0, 255, 0), 2)
        if jersey_number:
            cv2.putText(img0, f'#{jersey_number}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            print(f"✅ Tespit: {jersey_number}")

# Görseli göster
img_rgb = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(12, 8))
plt.imshow(img_rgb)
plt.axis("off")
plt.title("Detected Players and Jersey Numbers")
plt.show()

# Kaydet
cv2.imwrite("output_detected2.jpg", img0)
